In [84]:
from creds import (
    FOLDER_ID,
    OAUTH_TOKEN
)

import requests
import json
import pandas as pd
import time
import pickle

### connect to GPT

In [136]:
request = requests.post("https://iam.api.cloud.yandex.net/iam/v1/tokens", 
              json={"yandexPassportOauthToken" : OAUTH_TOKEN}, 
              )

IAM_TOKEN = request.json()['iamToken']
expiration_str = request.json()['expiresAt']
print("Token expires at", expiration_str[:10], expiration_str[11:16])

Token expires at 2024-03-27 20:06


In [110]:
instruction_text = f"""Ты работаешь продуктовым аналитиком в фаст-фуд компании. 
                    Выдели топ 3 конкретных рекомендаций по улучшению мобильного приложения для заказа еды на основе корпуса отзывов. 
                    Объясни подробно каждую данную тобой рекомендацию - почему она была тобой дана, на основе чего."""

In [111]:
request_text = """Не рекомендую заказывать через данное приложение, оформлял заказ, в итоге через полтора часа, в приложении срок доставки постоянно увеличивается; 
Не было ни одного раза когда при доставке положили все позиции, ни разу Даже при заказе в заведении не доклывают,отвратительно; 
Хочу оставить негативный отзыв, курьеры компании не доставили заказ, чеки все есть. Пришел даже разбирать в заведение где мой заказ, на что был культурно послан. А про поддержку я вообще молчу настолько проблемное приложение и люди в поддержке. Требовал вернуть средства уже неделю не читают сообщения.; 
Воруют деньги, если заказал не в тот ресторан. аргументируя тем, что срок годности истекает через 15 минут. Также сначала когда выбираю что-то из акции оно спрашивает, заказать ли по ближайшему адресу. Я нажимаю ага, а на деле оказывается адрес остаётся предыдущий и я заказываю не в тот ресторан. Недружелюбное поведение приложения зашкаливает; 
Приложение хорошее но, как это нет доставки на дом приходится заказывать и идти в вкусно-и точка и его забирать; Ужасный сервис доставки принимают заказ потом мурыжат пол часа и отменяют; Заказать доставку невозможно. По номеру горячей линии бот сбрасывает, в тех. поддержке игнор.; Взяли и не доложили 4 пирожка >:(; 
Доставка отвратительная. Хочу визу хочу нет; 
Нет доставки. Заказ только в сам ресторан.; 
Сделал заказ с доставкой на дом,деньги списали, заказа нет,на горячей линии телефон занят,деньги вернули спустя 10 дней.; 
По доставке не работает контакт центр, даже если, что то не довезли, как это обычно, обратится и решить проблему не где. Телефон не работает, на сообщение не отвечают .; Заказал доставку, доставки не было, деньги не вернули.; 
Заказала через приложение, через полчаса зашла посмотреть, когда привезут, а там написано: заказ аннулирован. В горячей линии сказали, что не хватило курьера. То есть нормально, что я жду заказ, а его просто так в любой момент могут аннулировать. В самом приложении не было ни бонусов, ни извинений, ни объяснения причины. Браво! Советую заказывать не через это приложение; 
Убрали доставку с другиз приложений, но при этом в своём не чего не до работали отменить заказ на стадии готовки нельзя, сменить адрес нельзя. Обратной связи нет. На звонки не ктотне отвечает."""

In [112]:
json_data_new = {
"model": "general",
"instruction_text": instruction_text,
"request_text": request_text,
"generation_options": {
                "max_tokens": 1500,  
                "temperature": 0.6
                }
}

In [113]:
json_file = '../data/prompt.json'

with open(json_file, "w") as fp:
    json.dump(json_data_new , fp)

with open(json_file) as json_data:
    prompt_date = json.dumps(json.load(json_data))

In [114]:
response = requests.post(
    "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct", 
    data=prompt_date,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {IAM_TOKEN}",
        "x-folder-id": f"{FOLDER_ID}",
    },
)

In [116]:
print(response.json()['result']['alternatives'][0]['text'])


1.	Сделать более наглядный интерфейс, который бы позволял пользователям, не имеющим опыта использования мобильных приложений, быстро разобраться в функционале и сделать заказ.
2.	Добавить возможность отслеживать статус заказа в режиме реального времени. Пользователи должны получать информацию о статусе заказа в виде уведомлений или чата с оператором.
3.	Улучшить систему лояльности для клиентов. Добавить возможность накапливать бонусы или скидки за каждый заказ. Это позволит привлечь новых клиентов и удержать старых.
4.	Внедрить функционал быстрой оплаты заказа. Клиенты должны иметь возможность оплатить заказ онлайн, чтобы избежать ожидания курьера с наличными.
5.	Устранить проблемы с доставкой. Обеспечить своевременное прибытие курьера и доставку заказа в указанное место.
6.	Расширить список ресторанов, которые можно заказать через мобильное приложение.
7.	Улучшить дизайн приложения. Сделать его более привлекательным и современным.
8.	Добавить функцию обратной связи. Дать возможность к

### Recommendations for scored data

In [132]:
sub_df = pd.read_csv('../data/scored/rostics.csv')

In [165]:
sub_df.head(1)

,Unnamed: 0,date,review,rating,review_processed,preds,type_preds
0,2,2024-03-03 18:57:45,Закал через это приложение. Ожидал в течение ч...,1,закал это приложение ожидал течение часа стату...,0,Функционал приложения


In [166]:
print(';\n'.join(sub_df[sub_df['type_preds'] == test_categories[0]]['review'].values[:1]))

Закал через это приложение. Ожидал в течение часа, статус заказа был «готовится», по итогу заказ был отменен самостоятельно. Ужасно.


In [135]:
sub_df.loc[
    (sub_df.type_preds!='Неинформативно')
    ,
    ['type_preds', 'rating']
    ].groupby('type_preds').count().sort_values('rating')



,rating
type_preds,
Регистрация в приложении,1
Локация ПБО,7
Возврат денег/Отмена заказа,22
Доставка,46
Функционал приложения,109


In [158]:
with open('../data/scored/recommendations.pickle', 'rb') as f:
    recommendations = pickle.load(f)

In [155]:
for business in [
    "rostics",
    "burger_king",
    # "vit",
    # "dodo",

]:
    # upload reviews
    sub_df = pd.read_csv(f'../data/scored/{business}.csv')

    # find categories of reviews
    test_categories = list(set(sub_df.type_preds.unique().tolist()) - set(['Неинформативно']))

    recommendations[business] = dict()

    # for each category of reviews get recommendations
    for test_category in test_categories:
        reviews = sub_df[sub_df['type_preds'] == test_category]['review'].values

        # if there are few reviews in this category 
        # we will not extract good recomendations
        if len(reviews) > 2:
            request_text = ';\n'.join(reviews[:25]) # 45 отзывов максимум

            json_data_new = {
                "model": "general",
                "instruction_text": instruction_text,
                "request_text": request_text,
                "generation_options": {
                "max_tokens": 1500,  
                "temperature": 0.6
                }
            }

            # get request in correct format
            json_file = '../data/prompt.json'
            with open(json_file, "w") as fp:
                json.dump(json_data_new , fp)

            with open(json_file) as json_data:
                prompt_date = json.dumps(json.load(json_data))

            # ------------------------------------

            response = requests.post(
                "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct", 
                data=prompt_date,
                headers={
                    "Content-Type": "application/json",
                    "Authorization": f"Bearer {IAM_TOKEN}",
                    "x-folder-id": f"{FOLDER_ID}",
                },
            )

            print(f'Категория: {test_category}')
            print(response.json()['result']['alternatives'][0]['text'])
            print('-'*10)
            print()
            time.sleep(5)
            recommendations[business][test_category] = response.json()['result']['alternatives'][0]['text']


# повторный заказ
# сохранение любимых ресторанов
# подключение соц. сетей
# регистрация через QR Code
# навигация - поиск товаров

Категория: Функционал приложения
1.	Сделайте более понятную навигацию по приложению и добавьте функцию сохранения данных в браузере, чтобы не вводить данные каждый раз при входе в приложение.
2.	Улучшите стабильность приложения, чтобы оно работало без сбоев и ошибок.
3.	Добавьте возможность выбора способа оплаты, например, наличными или картой, чтобы пользователи могли выбирать наиболее удобный для них способ оплаты.
4.	Упростите процедуру авторизации и регистрации, чтобы пользователи могли быстро и легко создавать аккаунты и входить в приложение.
5.	Увеличьте скорость загрузки страниц, чтобы пользователи не ждали долго перед оформлением заказа.
6.	Добавьте функцию поиска по меню, чтобы пользователи могли легко находить нужные им блюда.
7.	Улучшите дизайн приложения, чтобы оно выглядело более привлекательно и современно.
8.	Добавьте уведомления о скидках и акциях, чтобы пользователи были в курсе всех выгодных предложений.
9.	Улучшите работу с картой и поиском места на карте, чтобы поль

In [156]:
with open('../data/scored/recommendations.pickle', 'wb') as f:
    pickle.dump(recommendations, f)